Nantucket.ipynb, the interactive command-centre notebook: when run, the notebook reads the single YAML manifest that captures the Moby-Dick-themed directory plan, creates any folders and placeholder notebooks that are missing so your local OneDrive copy always matches that plan, checks that everything on disk still lines up with the manifest, and then regenerates human-readable documentation—a Markdown outline that includes an auto-rendered Mermaid diagram—so the project structure appears clearly in both GitHub (where the Markdown is rendered) and OneNote (with either the YAML or the diagram for easy reference). Because the whole Leviathan repository is cloned into Databricks Repos, Nantucket.ipynb travels with it; open the notebook inside an Azure Databricks workspace, run the same cells to scaffold or validate the structure there, and commit any changes straight back to GitHub. Edit the YAML manifest locally in JupyterLab, directly in Databricks, or in any text editor, rerunning Nantucket keeps every environment—OneDrive, GitHub, and Databricks—in lockstep.


In [12]:
# ── Cell 1 ── initialise structure + scaffold folders/notebooks
# ── Cell: Initialise manifest & scaffold project folders ──
import yaml, json, pathlib

NOTEBOOK_DIR = pathlib.Path.cwd()            # Leviathan/Nantucket
ROOT         = NOTEBOOK_DIR.parent           # Leviathan
MANIFEST     = NOTEBOOK_DIR / "leviathan_directory_manifest.yml"

# Your Moby-Dick–inspired directory plan
structure = {
    "Leviathan": {
        "type": "directory",
        "metaphor": "Leviathan",
        "role": "Project root",
        "notes": "",
        "children": {
            "Pequod": {
                "type": "directory",
                "metaphor": "Pequod",
                "role": "Docs & notebooks hub",
                "notes": "",
                "children": {
                    "Chartroom": {"type": "directory","metaphor": "Chartroom","role": "High-level overviews + config","notes": "","children": {}},
                    "Maps":      {"type": "directory","metaphor": "Maps",     "role": "Architecture diagrams + specs",  "notes": "","children": {}},
                    "Logbook":   {"type": "directory","metaphor": "Logbook",  "role": "Project diary & notes",        "notes": "","children": {}},
                },
            },
            "Ocean": {
                "type": "directory",
                "metaphor": "Ocean",
                "role": "Data lake",
                "notes": "",
                "children": {
                    "England_1": {"type": "directory","metaphor": "England_1","role": "Premier League data",    "notes": "","children": {}},
                    "England_2": {"type": "directory","metaphor": "England_2","role": "Championship data",      "notes": "","children": {}},
                },
            },
            "Armory": {
                "type": "directory",
                "metaphor": "Armory",
                "role": "Feature-engineering pipelines",
                "notes": "",
                "children": {
                    "Gam":       {"type": "directory","metaphor": "Gam",       "role": "Cross-league utilities",  "notes": "","children": {}},
                    "England_1": {
                        "type": "directory","metaphor": "England_1 (Armory)","role": "League-specific transforms","notes": "",
                        "children": {
                            "Spouter_inn": {"type": "directory","metaphor": "Spouter_inn","role": "Head-to-head & tables","notes": "","children": {}},
                            "Crows_nest":  {"type": "directory","metaphor": "Crows_nest", "role": "Rolling-form metrics",   "notes": "","children": {}},
                            "Whaleboat":   {"type": "directory","metaphor": "Whaleboat",  "role": "Expected-points models", "notes": "","children": {}},
                            "Tackle":      {"type": "directory","metaphor": "Tackle",     "role": "Composite rankings",     "notes": "","children": {}},
                        },
                    },
                },
            },
        },
    }
}

def _ensure_ipynb(path: pathlib.Path):
    """Make an empty notebook if missing."""
    if path.exists(): return
    empty = {"cells": [], "metadata": {}, "nbformat": 4, "nbformat_minor": 5}
    path.write_text(json.dumps(empty, indent=2))

def _create(node: dict, base: pathlib.Path):
    """Recursively mkdir and touch placeholder notebooks/files."""
    for name, meta in node.get("children", {}).items():
        p = base / name
        if meta["type"] == "directory":
            p.mkdir(parents=True, exist_ok=True)
            _create(meta, p)
        elif meta["type"] == "file":
            if p.suffix == ".ipynb":
                _ensure_ipynb(p)
            else:
                p.parent.mkdir(parents=True, exist_ok=True)
                p.touch(exist_ok=True)

# 1. Write the manifest into Nantucket/
MANIFEST.write_text(yaml.safe_dump(structure, sort_keys=False))

# 2. Scaffold everything under the true project root
_create(structure["Leviathan"], ROOT)

print("✅ Manifest written to", MANIFEST)
print("✅ Folders created under", ROOT)




✅ Manifest written to C:\Users\mikek\OneDrive\Leviathan\Nantucket\leviathan_directory_manifest.yml
✅ Folders created under C:\Users\mikek\OneDrive\Leviathan


In [13]:
# ── Cell 2 ── quick validation check (run anytime)
import yaml, pathlib

NOTEBOOK_DIR = pathlib.Path.cwd()                      # Leviathan/Nantucket
ROOT         = NOTEBOOK_DIR.parent                     # Leviathan
MANIFEST     = NOTEBOOK_DIR / "leviathan_directory_manifest.yml"

# Load the manifest from Nantucket/
manifest = yaml.safe_load(MANIFEST.read_text())["Leviathan"]

def _validate(node: dict, base: pathlib.Path) -> bool:
    ok = True
    for name, meta in node.get("children", {}).items():
        path = base / name
        if meta["type"] == "directory":
            if not path.is_dir():
                print(f"❌ missing folder {path.relative_to(ROOT)}")
                ok = False
            ok &= _validate(meta, path)
        elif meta["type"] == "file":
            if not path.is_file():
                print(f"❌ missing file {path.relative_to(ROOT)}")
                ok = False
    return ok

passed = _validate(manifest, ROOT)
print("✅ structure matches manifest." if passed else "⚠️ mismatch detected.")


✅ structure matches manifest.


In [14]:
# ── Cell 3 ── generate docs (outline + table + Mermaid)
import yaml, pathlib, textwrap, json

NOTEBOOK_DIR = pathlib.Path.cwd()        # Leviathan/Nantucket
ROOT = NOTEBOOK_DIR.parent               # Leviathan
MANIFEST = NOTEBOOK_DIR / "leviathan_directory_manifest.yml"
DOCS_DIR = NOTEBOOK_DIR                  # still write the docs here
DOCS_DIR.mkdir(exist_ok=True)

manifest  = yaml.safe_load(MANIFEST.read_text())["Leviathan"]

# -------------------------------- helpers --------------------------------
def flatten(node, base=""):
    rows = []
    for name, meta in node.get("children", {}).items():
        path = f"{base}/{name}".lstrip("/")
        rows.append((path, meta["metaphor"], meta["role"], meta["notes"]))
        rows += flatten(meta, path)
    return rows

rows = flatten(manifest)

# 1. Markdown outline
outline_lines = ["# Leviathan Directory Map\n"]
for path, metaphor, role, notes in rows:
    depth = path.count("/")
    indent = "  " * depth
    outline_lines.append(f"{indent}- **{path.split('/')[-1]}/** – {role}  ")
DOCS_DIR.joinpath("directory_map.md").write_text("\n".join(outline_lines))

# 2. Markdown table with Mermaid diagram
table_lines = [
    "| Path | Metaphor | Intended Role | Notes |",
    "|------|----------|---------------|-------|",
] + [
    f"| {p}/ | {m} | {r} | {n} |" for p, m, r, n in rows
]
mermaid = ["```mermaid", "graph TD"]
for p, m, _, _ in rows:
    parts = p.split("/")
    if len(parts) == 1:
        continue
    parent = parts[-2] or "Leviathan"
    mermaid.append(f"  {parent.replace('/', '_')} --> {parts[-1].replace('/', '_')}")
mermaid.append("```")

DOCS_DIR.joinpath("directory_glossary.md").write_text(
    "\n".join(table_lines) + "\n\n" + "\n".join(mermaid)
)

print("📝  docs/directory_map.md and docs/directory_glossary.md regenerated.")


📝  docs/directory_map.md and docs/directory_glossary.md regenerated.


In [15]:
mmd_path = NOTEBOOK_DIR / "directory_diagram.mmd"
mmd_path.write_text("\n".join(mermaid[1:-1])) 

258